In [1]:
!pip install transformers
!pip install accelerate datasets peft bitsandbytes sentencepiece loralib

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig,get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text,AutoProcessor,Trainer,TrainingArguments,BitsAndBytesConfig
from torchvision import transforms as transforms


In [6]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
checkpoint="HuggingFaceM4/idefics-9b"

In [7]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_skip_modules=["lm_head","embed_token"]
)

In [ ]:
processor=AutoProcessor.from_pretrained(checkpoint)

In [ ]:
model=IdeficsForVisionText2Text.from_pretrained(checkpoint,quantization_config=bnb_config,device_map="auto")

In [ ]:
#Inference
def do_inference(model, processor, prompts,max_new_token=50):
    tokenizer=processor.tokenizer
    bad_words=["<image>","<fake_token_around_image>"]
    if len(bad_words)>0:
       bad_words_ids=tokenizer(bad_words,add_special_tokens=False).input_ids
    eos_token="</s>"
    eos_token_id=tokenizer.convert_tokens_to_ids(eos_token)
    
    inputs=processor(prompts,return_tensors="pt").to(device)
    generated_ids=model.generate(
      **inputs,
      eos_token_id=[eos_token_id],
      bad_words_ids=bad_words_ids,
      max_new_tokens=max_new_token,
     early_stopping=True,
    )
    generated_text=processor.batch_decode(generated_ids,skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text







           

In [ ]:
##preprocessing
def convert_to_rgb(image):
    if image.mode=="RGB":
        return image
    image_rgba=image.convert("RGBA")
    background=Image.new("RGBA",image_rgba.size,(255,255,255))
    alpha_composite=Image.alpha_composite(background,image_rgba)
    alpha_composite=alpha_composite.convert("RGB")
    return alpha_composite

def ds_transform(example_batch):
    image_size=processor.image_processor.image_size
    image_mean=processor.image_processor.image_mean
    image_std=processor.image_processor.image_std

    image_transform=transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size,image_size),scale=(0.9,1.0),interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean,std=image_std)
    ])

    prompts=[]
    for i in range(len(example_batch["caption"])):
        caption=example_batch['caption'][i].split(".")[0]
        prompts.append([
           example_batch['image_url'][i],
           f"Question: What's on the picture? Answer: This is {example_batch['name']}.{caption}</s>"
        ])   
    inputs= processor(prompts,transform=image_transform,return_tensors="pt").to(device)
    inputs["labels"]=inputs["input_ids"]
    return inputs   

In [ ]:
ds=load_dataset("TheFusion21/PokemonCards")
ds=ds["train"].train_test_split(test_size=0.002)
train_ds=ds["train"]
evals_ds=ds["test"]
train_ds.set_transform(ds_transform)
evals_ds.set_transform(ds_transform)

In [ ]:
model_name=checkpoint.split("/")
config=LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj",
    "k_proj",
    "v_proj"
    ],
    lora_dropout=0.01,
    bias="none",
   
)

In [ ]:
model=get_peft_model(model,config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
training_args=TrainingArguments(
    output_dir=f"{model_name}-Pokemon",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=True,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=25,
    eval_steps=10, 
    max_steps=25,
    logging_steps=10,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=[labels],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    optim="paged_adamw_8bit",
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=evals_ds,
)

In [ ]:
trainer.train()

In [ ]:
url="https://images.pokemontcg.io/pop6/2_hires.png"

In [ ]:
promopts=[
    url,
    "Question: What's on the picture? Answer: ",
]

In [ ]:
do_inference(model,processor,promopts,max_new_token=100)

In [ ]:
import locale
locale.getpreferredencoding=lambda: "utf-8"